# Layers

本文翻译自：http://implicit-layers-tutorial.org/introduction/

## Introduction: Explicit layers in deep learning

现代深度学习方法的核心是*层*的概念。深度学习模型传统上是通过将许多这些层堆叠在一起来构建的，以创建旨在解决某些特定任务的架构。例如，卷积网络由卷积层组成，通常遵循像 ReLU 那样的元素非线性，具有归一化或 dropout 等附加操作，并且可能以多种不同的方式连接在一起，形成残差层之类的东西。同样，像 Transformer 网络这样的架构由所谓的自注意力self-attention层和全连接层的组合组成，它们再次以某种方式堆叠在一起，并最终形成模型。

一个共同的定义的特征（这一点是如此标准以致于从业者经常忽视）是现代深度学习中的这些层中的绝大多数都是明确定义的。也就是说，它们由从输入到输出层执行的精确运算序列指定。让我们以一个扩展的自我注意层为例 [[Vaswani等](https://arxiv.org/abs/1706.03762)]。这一层是三个矩阵$K,Q,V \in \mathbb{R}^{T \times n}$到$Z \in \mathbb{R}^{T \times n}$的映射 ，由以下运算定义

$Z = \mathrm{SelfAttention}(K,Q,V) \equiv \mathrm{softmax}\left (\frac{K Q^T}{\sqrt{n}} \right) V$

这是self-attention的简化版，仅作说明，没有masks 或 multi-head 结构。我们可以将此层编写为一个简单的 Python 函数（同样，只是为了说明，而不是实际想要编写的方式，例如 softmax 操作，更不用说可能想要使用自动微分库而不是简单的numpy 来编写这些函数）。

In [1]:
import numpy as np

def self_attention(K,Q,V):
    A = np.exp(K @ Q.T) / np.sqrt(K.shape[1])
    return (A / np.sum(A,1)) @ V

K, Q, V = np.random.randn(3, 5, 4)
print(self_attention(K, Q, V))

[[ -0.28216079   0.05426659  -0.04054614  -0.29725411]
 [-10.48530329  -2.28341585   6.06080698 -13.22596288]
 [ -5.03490353   0.43058437   2.32202246  -5.77957903]
 [ -1.19333602  -0.11957199   0.73340623  -1.56589836]
 [ -0.53730425  -0.06265875   0.27005192  -0.73109501]]


自然地，随着我们向层本身添加更多功能，在自动微分库中实现它们等，事情开始变得更加复杂，但是典型层的这种显式形式仍然贯穿其中：层的构建很大程度上类似于典型的计算机程序，我们直接编写代码来生成层的输出作为其输入的函数。事实上，这可能是如此根深蒂固，以至于很难想象有一种完全不同的方式可以定义层，即通过隐式层。

## Implicit layers

正如我们将在整个文档中使用的术语一样，隐式层的关键在于，不是指定如何从输入计算层的输出，而是指定我们希望层的输出满足的条件。也就是说，如果我们要编写显式层（带有输入$x \in \mathcal{X}$ 和输出 $z \in \mathcal{Z}$) 作为一些显式函数的应用 $f : \mathcal{X} \rightarrow \mathcal{Z}$

$z = f(x)$

然后将通过函数 $g : \mathcal{X} \times \mathcal{Z} \rightarrow \mathbb{R}^n$ 定义隐式层，这是x  和 z两者的联合函数，其中，z层的输出 需要满足一些约束，例如，找到方程的根，

$\mbox{Find $z$ such that } g(x,z) = 0.$

这里的符号可能表明g(x,z)是一个简单的代数方程，但在实践中，同样的形式可以：

- algebraic equations and fixed points -> recurent backprop models or deep equilibrium models；
- 微分方程 -> Neural ODEs；
- the optimality conditions of optimization problems -> differentiable optimization approaches

在进入一个具体的例子之前，一个事实是，最初转向这个隐含的公式似乎是一个微不足道的点。毕竟，为了实际实现这样的一层，我们需要指定某种实际计算方程式g的根的方式。但正如很快就会看到的，考虑层的隐式形式有许多实际优势。

从根本上说，隐式层将层的求解过程与层本身的定义分开 *separate the solution procedure of the layer from the definition of the layer itself*。事实证明，这种级别的模块化在许多领域都非常有用。例如，试图找到常微分方程的数值解的微分方程求解器可以实现各种自适应步长、所谓的“刚性”方程的修正等，所有这些都是为了试图找到一个低可微方程的误差解。另一个示例，优化求解器通常涉及解决某些类型问题的非常复杂的启发式方法，但它们的全部目的都是为优化任务找到最小目标的解决方案。确实，因为我们很少找到准确的代数或微分方程的解，不同的解法可以根据它们满足层试图满足的条件的程度来客观地相互评估。

这种层的目标与其求解方法的分离本身就足够令人满意，但隐式层的第二个优势特别出现在深度学习和自动微分的背景下。机器学习中自动微分 (AD) 的传统方法是在自动微分框架（例如 PyTorch、Tensorflow 或 JAX）中实现所有层，这让我们可以立即将这些层包含在需要梯度来拟合模型到数据的深度模型中。然而，直接在 AD 库中实现求解程序，尤其是那些涉及迭代更新的程序，如标准微分方程或优化求解器，意味着我们需要存储完整求解程序的计算图，以及在此解决方案期间创建的临时迭代的值。这需要在内存中存储大量信息，这在训练大型深度学习模型时通常会成为瓶颈。幸运的是，正如我们将在下面说明的那样，并在本教程中进行了多次强调，隐式层具有明显的优势，即我们可以使用隐函数定理直接计算这些方程的解点处的梯度，而不必沿途存储任何中间变量 *we can use the implicit function theorem to directly compute gradients at the solution point of these equations*。这极大地提高了这些方法的内存消耗和数值准确性，特别是在深度学习的设置中，为隐式模型提供了另一个显著的好处。

### 应用和说明

本文剩余部分将重点放在一个极其简单的演示上，该演示旨在作为方法的教学说明（而不是对最先进性能的说明），以简要强调使用隐式层。以下只是一小部分样例（原文作者最熟悉的实例），但希望能够说明当前隐式层研究解决方法的广度。将在文本中深入探讨其中更多细节，但了解这些方法所解决的所有应用领域要进一步查看该领域的当前研究。

隐式层已被用于：

- cvxpy以可微分的方式解决任意结构化的凸问题（使用库）。
- 解决组合优化问题的平滑松弛，例如图切割、可满足性等。
- 将微分方程整合为深层网络中的层（其本身有许多应用，例如整合连续时间观察，或逼近传统残差网络的连续版本）。
- 创建用于有效表示平滑密度的架构，用于生成式建模及其他。
- 实现与最先进的 Transformer 模型（在相同的参数计数下）、语言建模以及在分类和语义分割等任务上与最先进的计算机视觉架构相媲美的性能。

## Outline of this work

以上述简要介绍为背景，概述除本文外的其余部分以及其余各篇文章如何组合在一起。

- 在本文的剩余部分，简要介绍通过定点迭代定义的第一个隐式层。这本质上是循环反向传播的一个版本，它是隐式层的最早形式之一，可以追溯到 80 年代后期，也是深度均衡 (DEQ) 模型的基础方法。
- 在第 2 章中，我们将讨论隐式模型背后的数学背景，包括隐函数定理及其在自动微分工具中的实现。（备注：都是数学推导，暂略）
- 在第 3 章中，我们将介绍神经 ODE，这是近年来受到广泛关注的隐式层的实例。我们将介绍基本的数学框架并重点介绍该模型的一些应用。（用到了jax，本repo是在windows下运行的，jax在windows下不好安装，所以需在win10下的Ubuntu里跑，因而放到[elks中](https://github.com/OuyangWenyu/elks/tree/master/math-basics/equation)了）
- 在第 4 章中，我们将更详细地介绍深度均衡模型，重点是将本文中介绍的基本思想扩展到现代深度学习框架，并强调模型的一些正在进行的方向和应用。
- 在第5章中，我们将介绍可微分的优化，将优化问题的解决方案作为层级嵌入。

## Your first implicit model: fixed point iteration

在深入研究数学细节和许多不同形式的隐式模型之前，让我们从一个特别简单的例子开始：由定点迭代定义的网络层。如上所述，这种类型的层可以追溯到循环反向传播的一些原始公式，也是我们很快将讨论的深度均衡模型的基础。

### A fixed point iteration layer

虽然我们很快就会采用这个层是特定方程的根的观点，但为了介绍这个层，假设我们有相同维度的输入和输出 $x,z \in \mathbb{R}^n$ ，并考虑以下计算输出z 作为x的函数的方法，网络参数 $W \in \mathbb{R}^{n \times n}$

$\begin{split}
& z := 0 \\
& \mbox{Repeat until convergence:} \\
& \quad z := \tanh(Wz + x)
\end{split}$

这是定点迭代fixed point iteration的一个实例：在某些条件下，该过程将收敛到某个固定输出$z^\star$。所以显然它具有以下性质：

$z^\star = \tanh(W z^\star + x)$

我们现在将推迟讨论为什么这可能是一个特别好的层应该采用的形式，但简单地说，这种类型的层可以被解释为一个简单的循环网络，其中z是隐藏层，我们将重复应用这个网络于相同的输入X。像这样的层也可以，例如，获得“深度”神经网络的一些好处（因为它涉及非线性的重复应用），而只有参数宽“单”层。但是我们将讨论这些优势层，现在只关注简单地使用这样的层。

注意，当然这可以写成上面形式的隐式层，即 z⋆ 是求根方程的解

$\mbox{Find $z$ such that } g(x,z) = 0, \quad \mbox{where } g(x,z) \equiv z - \tanh(W z + x)$

请注意，此迭代实际上不需要收敛：尽管 tanh 激活将强制 z值 永远不离开 [−1 , +1]区间，但它取决于W值，可能是值无休止地循环并且永远不会达到固定点的。另一方面，例如 W= 0，然后经过一次迭代，到达一个“固定点” z⋆= tanh（x ）。我们在这里要说的是，对于“典型”W值 （读作：大多数深度学习库使用的线性层的默认值，加上它们在优化时达到的值），这一迭代确实会收敛，稍后会讨论fixed点的存在性和唯一性问题。

### Implementing a fixed point iteration layer

与传统层相比，在 PyTorch 或 JAX 等自动微分库中实现隐式层肯定需要更多的努力。但是实现的实际核心仍然非常简单，并且通过这些工具仍会变得更加容易。

首先，让我们考虑像这样的层的最简单可能的实现，它只是通过库的正常 autograd 功能重复定点迭代以收敛（即，我们只是“展开”定点计算）。因为这是通过正常的 autograd 机制发生的，每个中间迭代都必须存储在内存中，并且向后传递必须在相同的迭代中类似地进行，但顺序相反。现在，我们将制作一个单独的层，它完全实现了tanh 和上面的线性层（加上其他简单的技巧，例如存储最近的迭代计数和误差），但在后面的章节中，我们将使其更加模块化，以便我们可以找到使用相同的库实现的通用层找到类似固定点

In [2]:
import torch
import torch.nn as nn

class TanhFixedPointLayer(nn.Module):
    def __init__(self, out_features, tol = 1e-4, max_iter=50):
        super().__init__()
        self.linear = nn.Linear(out_features, out_features, bias=False)
        self.tol = tol
        self.max_iter = max_iter
  
    def forward(self, x):
        # initialize output z to be zero
        z = torch.zeros_like(x)
        self.iterations = 0

        # iterate until convergence
        while self.iterations < self.max_iter:
            z_next = torch.tanh(self.linear(z) + x)
            self.err = torch.norm(z - z_next)
            z = z_next
            self.iterations += 1
            if self.err < self.tol:
                break

        return z

我们可以在随机输出上运行这个层，看它实际上确实到达了一个固定点。

In [3]:
layer = TanhFixedPointLayer(50)
X = torch.randn(10,50)
Z = layer(X)
print(f"Terminated after {layer.iterations} iterations with error {layer.err}")

Terminated after 14 iterations with error 7.896462921053171e-05


虽然简单地运行在随机数据层到最后会不那么翔实，但是我们可以看看在一个真实模型中使用它，会更有趣。因此，下面展示了一个在 MNIST 数据集上训练的简单模型，使用一个fixed point layer（在定点层之前有一个额外的线性输入层，在定点层之后有一个线性层）。该模型无意打破这里的任何记录，但它提供了一个稍微有用的示例，可以在此基础上进行进一步的实验，而不仅仅是单独运行该层。

In [4]:
# import the MNIST dataset and data loaders
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

mnist_train = datasets.MNIST("data/mnist", train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST("data/mnist", train=False, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(mnist_train, batch_size = 100, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size = 100, shuffle=False)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
# construct the simple model with fixed point layer
import torch.optim as optim

torch.manual_seed(0)
model = nn.Sequential(nn.Flatten(),
                      nn.Linear(784, 100),
                      TanhFixedPointLayer(100, max_iter=200),
                      nn.Linear(100, 10)
                      ).to(device)
opt = optim.SGD(model.parameters(), lr=1e-1)

In [6]:
# a generic function for running a single epoch (training or evaluation)
from tqdm.notebook import tqdm

def epoch(loader, model, opt=None, monitor=None):
    total_loss, total_err, total_monitor = 0.,0.,0.
    model.eval() if opt is None else model.train()
    for X,y in tqdm(loader, leave=False):
        X,y = X.to(device), y.to(device)
        yp = model(X)
        loss = nn.CrossEntropyLoss()(yp,y)
        if opt:
            opt.zero_grad()
            loss.backward()
            if sum(torch.sum(torch.isnan(p.grad)) for p in model.parameters()) == 0:
                opt.step()
        
        total_err += (yp.max(dim=1)[1] != y).sum().item()
        total_loss += loss.item() * X.shape[0]
        if monitor is not None:
            total_monitor += monitor(model)
    return total_err / len(loader.dataset), total_loss / len(loader.dataset), total_monitor / len(loader)

让我们最终将模型训练 10 个 epoch。除了训练/测试错误/损失之外，我们还将打印出层收敛到固定点所需的固定点迭代的平均次数。

In [7]:
for i in range(10):
    if i == 5:
        opt.param_groups[0]["lr"] = 1e-2

    train_err, train_loss, train_fpiter = epoch(train_loader, model, opt, lambda x : x[2].iterations)
    test_err, test_loss, test_fpiter = epoch(test_loader, model, monitor = lambda x : x[2].iterations)
    print(f"Train Error: {train_err:.4f}, Loss: {train_loss:.4f}, FP Iters: {train_fpiter:.2f} | " +
          f"Test Error: {test_err:.4f}, Loss: {test_loss:.4f}, FP Iters: {test_fpiter:.2f}")

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Train Error: 0.1116, Loss: 0.4020, FP Iters: 54.14 | Test Error: 0.0672, Loss: 0.2232, FP Iters: 56.13


  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

Train Error: 0.0175, Loss: 0.0632, FP Iters: 79.46 | Test Error: 0.0306, Loss: 0.1031, FP Iters: 83.24


  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Train Error: 0.0174, Loss: 0.0616, FP Iters: 82.14 | Test Error: 0.0307, Loss: 0.1051, FP Iters: 87.91


这里并没有完全打破任何记录（单个隐藏层网络实现了相同的性能，执行/训练速度更快），但很好的是网络至少能使用这一层进行训练。不过，有一些注意事项。第一个是我们最终对相当多的层运行定点迭代，以便收敛到$10^{-4}$内的一个固定点。如果查看每个minibatch所需的单独迭代，会发现其中一些甚至根本没有达到此容差，但在 200 步后以较低的容差水平退出（固定点可能迭代甚至可能在训练过程中的某些时候变得不稳定，如果在没有适当的错误处理的情况下发生，通常会大大降低模型的性能）。这似乎是一个相当大的缺点。原文作者在实践中有效地运行了一个 50-80“层”的网络，并没有看到比标准 MLP 有多大优势（不过不是这样，因为在每次迭代$z := \tanh(Wz + x)$时将输入重新添加输入到层，这和传统神经网络的深度是不相同的，传统MLP会遭受梯度vashing /爆炸）。

为了真正看到这些层的潜在优势，需要引入更多的想法。

### Alternative root finding techqniues

回想一下，隐含层中的一个好处是，它分离了 层计算的是什么，和层如何计算。在上面的例子中，我们定点迭代的目标是找到一些z使得：

$z= tanh(Wz+ x)$

一种实现方法是简单地迭代此方程式，但这绝不是唯一的方法。或者，我们可以使用更快的寻根方法，例如牛顿法，以尝试更有效地找到解。

牛顿法是一种通用的根求解技术。对于某些函数$g : \mathbb{R}^n \rightarrow \mathbb{R}^n$, 如果我们想找到一个根 g（z）= 0，那么可以使用牛顿法重复更新：

$z := z - \left ( \frac{\partial g}{\partial z} \right ) ^{-1} g(z)$

其中，$\frac{\partial g}{\partial z}$ 表示f 关于 z的Jacobian （在实践中经常需要一个“受保护的”更新，它采取更小的步骤来确保残差∥g（z）∥的充分减少，但在这里不考虑这一点）。尽管我们可以求助于自动微分来计算雅可比式（将在后面的章节中，在定点迭代中使用更通用的层时这样做），对于我们的tanh加上线性层的情况，很容易以封闭形式计算雅可比式。具体来说，我们试图找到方程g( x , z）= 0的根（回到本章前几节的符号，在那里我们对层输入 X的依赖是 显式的），其中

$g(x,z) = z - \tanh(Wz + x)$

那么雅可比矩阵由下式给出

$\frac{\partial g}{\partial z} = I - \mathrm{diag}(\tanh'(Wz + x)) W$

其中，tanh′ 表示tanh 函数的导数，

$\tanh'(x) = \mathrm{sech}^2(x)$

让我们看看牛顿方法在代码中的实现是什么样的。由于需要计算牛顿步长，因此与简单的定点迭代相比，该实现涉及更多的事务，但这最终仅是几行附加的代码。

In [8]:
class TanhNewtonLayer(nn.Module):
    def __init__(self, out_features, tol = 1e-4, max_iter=50):
        super().__init__()
        self.linear = nn.Linear(out_features, out_features, bias=False)
        self.tol = tol
        self.max_iter = max_iter
  
    def forward(self, x):
        # initialize output z to be zero
        z = torch.tanh(x)
        self.iterations = 0
    
        # iterate until convergence
        while self.iterations < self.max_iter:
            z_linear = self.linear(z) + x
            g = z - torch.tanh(z_linear)
            self.err = torch.norm(g)
            if self.err < self.tol:
                break

            # newton step
            J = torch.eye(z.shape[1])[None,:,:] - (1 / torch.cosh(z_linear)**2)[:,:,None]*self.linear.weight[None,:,:]
            z = z - torch.solve(g[:,:,None], J)[0][:,:,0]
            self.iterations += 1

        g = z - torch.tanh(self.linear(z) + x)
        z[torch.norm(g,dim=1) > self.tol,:] = 0
        return z

In [9]:
layer = TanhNewtonLayer(50)
X = torch.randn(10,50)
Z = layer(X)
print(f"Terminated after {layer.iterations} iterations with error {layer.err}")

Terminated after 3 iterations with error 1.094531171474955e-06


这能够比不动点迭代更快地很好地收敛，但有一个（主要）警告，我们现在必须在每次迭代中求解线性系统。同样，由于我们使用自动微分来实现整个过程，这意味着我们还需要在反向传播中对这解进行反向传播。但是我们可以简单地将其插入到与以前相同的训练过程中。

In [10]:
torch.manual_seed(0)
model = nn.Sequential(nn.Flatten(),
                      nn.Linear(784, 100),
                      TanhNewtonLayer(100, max_iter=40),
                      nn.Linear(100, 10)
                      ).to(device)
opt = optim.SGD(model.parameters(), lr=1e-1)

for i in range(2):
    if i == 5:
        opt.param_groups[0]["lr"] = 1e-2

    train_err, train_loss, train_fpiter = epoch(train_loader, model, opt, lambda x : x[2].iterations)
    test_err, test_loss, test_fpiter = epoch(test_loader, model, monitor = lambda x : x[2].iterations)
    print(f"Train Error: {train_err:.4f}, Loss: {train_loss:.4f}, Newton Iters: {train_fpiter:.2f} | " +
          f"Test Error: {test_err:.4f}, Loss: {test_loss:.4f}, Newton Iters: {test_fpiter:.2f}")

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Train Error: 0.1145, Loss: 0.4120, Newton Iters: 6.75 | Test Error: 0.0636, Loss: 0.2213, Newton Iters: 6.79


  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

Train Error: 0.9065, Loss: 6.0890, Newton Iters: 23.84 | Test Error: 0.9061, Loss: 6.0984, Newton Iters: 21.90


  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Train Error: 0.9065, Loss: 6.0890, Newton Iters: 23.79 | Test Error: 0.9061, Loss: 6.0984, Newton Iters: 21.90


同样，该方法运行得相当好。但是，所实施的方法存在一些显着的问题。首先，如果运行代码，会立即注意到，该方法明显比上面更简单的定点迭代方法慢。尽管所需的迭代次数比定点迭代少得多，但每个单独的迭代也慢得多，因为它涉及minibatch中每个样本形成和求逆一个单独的（在这种情况下，100 × 100) 的雅可比矩阵。对于较大的隐藏单元大小（尤其是例如卷积网络），将这些矩阵求逆甚至存储将变得非常棘手。事实上，在实践中，很少使用精确的牛顿方法，相反，我们可以使用拟牛顿方法来改善标准定点迭代的收敛性，同时也改善运算时间。

这种方法的第二个问题有点微妙，但实际上是一个更大的问题。因为我们直接在自动微分工具包中实施了牛顿方法，所以这种方法有一些很大的缺点。首先，与定点迭代一样，自动微分工具需要保存隐藏单元的中间迭代；但是在这里，这意味着我们还需要在内存中存储Jacobian项的中间迭代，即使在可能的情况下，这也会大大增加内存消耗存储并求逆完整的雅可比行列式。此外，通过重复逆的反向传播可能是一个数值不稳定的过程：如果逆接近奇异，那么即使前向传播正确收敛，后向传播仍然会在梯度中产生数值误差。事实上，您会注意到我们在epoch()中包含了一个“NaN check”. 如果我们不这样做，那么对于 Newton 的方法，该方法将立即失败：如果检查，将看到大约 5% 的更新实际上在梯度中具有 NaN 值，这是由于雅可比行列式的条件，这也是导致该方法实际上比定点迭代版本收敛得慢的原因之一。

这为解决隐式模型的“高效”方法描绘了一幅相当模糊的图景。然而，幸运的是，由于隐函数定理，有一种更好的方法来实现这些层。

### Differentiation in implicit layers

到目前为止，我们以与实现任何其他层完全相同的方式为我们的隐式层实现了我们的求解器，并让自动微分库负责向后传递。然而，有一种更好的方法来区分隐藏层的不动点。为了了解如何做到这一点，让我们考虑隐式层的通用形式，即给定x，找到一些 z 使得

$g( x , z)= 0$

令 z⋆（x ） 作为解决这个不动点的值，这样写是为了强调隐式层的输出还是输入的（隐式）函数。

现在让我们考虑如何计算这个输出相对于输入的雅可比行列式

$$\frac{\partial z^\star(x)}{\partial x}$$

与习惯的传统函数不同，在传统函数中，我们给出了用于计算输入输出的显式形式，如何确定这样的雅可比行列式可能并不明显。但实际上，使用隐式微分计算该项非常简单，这种技术可以追溯到几个世纪的微积分。特别地，为了推导出这个雅可比矩阵的表达式，我们从不动点条件开始，我们知道它适用于z⋆(x)，并在两边对x微分 ,

$$\frac{\partial g(x,z^\star(x))}{\partial x} = 0.$$

现在我们只使用链式法则来扩展这个偏导数：因为 g 是两个变量的函数，将有一项涉及每个变量的导数：

$$\frac{\partial g(x,z^\star)}{\partial x} + \frac{\partial g(x,z^\star)}{\partial z^\star}\frac{\partial z^\star(x)}{\partial x} = 0$$

其中，符号 z⋆ （未表示为 X的函数), 只是表示我们正在将 z⋆作为这里的固定值（即雅可比$\frac{\partial g(x,z^\star)}{\partial x}$将只是在点 （x ，z⋆）处 g 关于 x的雅可比）。因此，该项与$\frac{\partial g(x,z^\star)}{\partial z^\star}$项，可以使用普通的自动微分库自行计算。最后，我们只需重写这个等式，根据我们知道的表达式给出我们所要求的表达式

$$\frac{\partial z^\star(x)}{\partial x} = - \left ( \frac{\partial g(x,z^\star)}{\partial z^\star} \right )^{-1} \frac{\partial g(x,z^\star)}{\partial x}$$

从技术上讲，为了确保我们可以实际应用该定理，我们要求必须满足某些条件，以便隐式函数 z⋆（x ）保证存在：这些条件反映在所谓的隐函数定理中，这将在下一章中讨论。此外，就像牛顿法与拟牛顿法一样，在实践中通常不可能直接计算这个逆，而是需要一个迭代过程。我们将在下一章中更多地介绍数学细节和形式主义，但就我们实际需要推导的大部分内容而言，这种“非正式”推导实际上就是所需要的。最后，虽然我们写了上面的关于X雅可比公式， 当 G 也是一些参数θ （例如，权重和偏差）的函数时 ，完全相同的推导适用于找到关于这些参数的雅可比式。

然而，从这个公式的详细推导回来，隐函数定理导致了一个非常实际的结果。即，该公式给出了必须的雅可比式的形式，而无需通过用于获得不动点的方法进行反向传播。换句话说，怎么计算函数的零点都无所谓（无论是通过定点迭代、牛顿法还是拟牛顿法）。重要的是找到不动点（使用您想要的任何技术），此时我们可以使用这种分析形式直接计算必要的雅可比式（或者更准确地说，计算向后传递，这通常不需要显式计算雅可比式）。用于计算不动点的迭代方法的中间项不需要存储在内存中（使方法的内存效率更高），并且不需要在自动微分层中展开前向计算。

### Implementing implicit differentiation

让我们看看隐微分的实现在实践中是如何工作的。首先，让我们再次考虑我们的 tanh 加线性层，其中g( x , z） 函数由下式给出

$$g(x,z) = z - \tanh(Wz + x)$$

在这种情况下，雅可比 $\frac{\partial g}{\partial z^\star}$，隐式微分所需，由下式给出

$$\frac{\partial g}{\partial z^\star} = I - \mathrm{diag}(\tanh'(Wz^\star+x)) W$$

你可能会注意到这就是我们在使用牛顿法求解不动点时形成的雅可比式。这并非偶然：实际上，在牛顿方法中寻找根所需的雅可比项与通过隐式微分计算向后传递完全相同。这产生了一个非常好的属性：对于这种情况，我们通过牛顿方法（或任何计算和求逆雅可比方法的方法）找到根的解，然后通过牛顿方法计算反向传递实际上是“自由的”（至少，相对于开始求解不动点的复杂性而言）：我们可以简单地重用我们在前向传递中所做的雅可比行列式（及其逆）。当然，由于在实践中我们经常使用拟牛顿法或一阶法来寻找隐含层的不动点，这并不像看起来那么大优点。但是，尽管如此，在我们做计算，即使在向前传递时的近似值雅可比，它都有利于在反向传播中充分利用这一计算。

在继续实现之前，应该强调实际的隐式微分过程在反向传播中是如何工作的（即，反向模式自动微分）。在反向传播中，我们实际上不需要计算网络中间层的完整雅可比行列式。相反，反向传播的目标是计算关于一些标量loss的梯度。如果我们根据上面的梯度写出它，它看起来像

$$\frac{\partial \ell}{\partial x} = \frac{\partial \ell}{\partial z^\star} \frac{\partial z^\star}{\partial x} = - \frac{\partial \ell}{\partial z^\star} \left (\frac{\partial g}{\partial z^\star} \right )^{-1} \frac{\partial g}{\partial x}$$

我们在最后一个等式中应用了上面的隐式微分公式。在反向传播中，该项是从左到右计算的，这意味着实际上不需要计算完整的雅可比行列式$\frac{\partial z^\star}{\partial x}$，我们只需要计算上面显示的向量雅可比乘积。按照惯例，大多数自动微分框架根据梯度运算（标量值函数的雅可比矩阵转置）来构建此框架

$$\nabla_{z^\star} \ell = \left ( \frac{\partial \ell}{\partial z^\star} \right )^T$$

所以我们需要乘以雅可比的转置

$$\nabla_x \ell = \left (\frac{\partial g}{\partial x} \right )^T \left (\frac{\partial g}{\partial z^\star} \right )^{-T} \nabla_{z^\star} \ell$$

然而，我们再次强调，我们实际上并不需要存储和计算实际的逆 $\left (\frac{\partial g}{\partial z^\star} \right )^{-T}$，只要能够解出这个公式中出现的（线性）方程。

最后，让我们讨论如何在自动微分工具包中实现这样的公式。细节当然会因框架而异，但由于我们最终在这里讨论的是实现一种新型功能（即，在前向传递中计算任何自动微分之外的固定点，然后计算“自定义”向后传递）您可能会想使用像autograd.Function接口这样的功能（例如，如果您在 PyTorch 中实现它），它允许您完全在库的正常自动微分传递之外指定向前和向后传递。但这实际上在实践中会有些麻烦：毕竟，自动微分的好处之一是我们可以潜在地实现函数g。 （无论是使用卷积、自注意力还是任何其他特征）在同一个自动微分库中，我们将自动包含所有这些梯度，而无需为每个特定函数编写我们要实现的新函数g 。幸运的是，有一种相当简单但微妙的方法来处理这个问题。我们将在后面的部分中返回几个有效的隐式微分示例，每个示例都有自己的实现怪癖，但对于像这样的简单示例，有效的通用范例是以下三个步骤：

1. 自动微分之外，求解隐含层g（x ，z⋆）= 0的根.
2. 通过在自动微分中运行以下分配来“重新参与”自动微分：$z := z^\star - g(x,z^\star)$，这具有“重新插入”偏导数 -∂g/∂x 到 autograd 中的效果（并且在价值方面是空的 ž，因为g（x ，z⋆）= 0 ）。
3. 向后传递添加一个“后向钩子”，乘以 $(\frac{\partial g}{\partial z^\star})^{-T}$. 这将修复向后传递，以便它根据隐函数定理正确实现梯度。

对于之前的 tanh + 线性层，这会导致如下实现。请注意，该层与我们之前实现的版本基本相同，只是 Newton 的方法在一个torch.no_grad():块内运行，并且我们通过该register_hook函数添加了向后传递钩子。对于上面的第二步，给定G 之前强调的功能，赋值很简单

$z := z^\star - g(x,z^\star) = z^\star - z^\star + \tanh(Wz + x) = \tanh(Wz^\star + x)$

即，在使用牛顿法找到不动点后，我们在自动微分带内运行单个不动点迭代。

In [11]:
class TanhNewtonImplicitLayer(nn.Module):
    def __init__(self, out_features, tol = 1e-4, max_iter=50):
        super().__init__()
        self.linear = nn.Linear(out_features, out_features, bias=False)
        self.tol = tol
        self.max_iter = max_iter
  
    def forward(self, x):
        # Run Newton's method outside of the autograd framework
        with torch.no_grad():
            z = torch.tanh(x)
            self.iterations = 0
            while self.iterations < self.max_iter:
                z_linear = self.linear(z) + x
                g = z - torch.tanh(z_linear)
                self.err = torch.norm(g)
                if self.err < self.tol:
                    break

                # newton step
                J = torch.eye(z.shape[1])[None,:,:] - (1 / torch.cosh(z_linear)**2)[:,:,None]*self.linear.weight[None,:,:]
                z = z - torch.solve(g[:,:,None], J)[0][:,:,0]
                self.iterations += 1
    
        # reengage autograd and add the gradient hook
        z = torch.tanh(self.linear(z) + x)
        z.register_hook(lambda grad : torch.solve(grad[:,:,None], J.transpose(1,2))[0][:,:,0])
        return z

请注意，这是一个非常不标准的实现：我们在常规自动微分带之外实现正向传递的元素，然后添加一个向后遮罩来“固定”渐变。我们可以使用buildinggradcheck命令来验证这一层的正确性。请注意，这种实现不适用于双重反向传播（即，gradgradcheck不会起作用），但这可以通过稍微复杂的方法解决，并且在实践中通常不需要，所以我们现在忽略它。

In [12]:
from torch.autograd import gradcheck

layer = TanhNewtonImplicitLayer(5, tol=1e-10).double()
gradcheck(layer, torch.randn(3, 5, requires_grad=True, dtype=torch.double), check_undefined_grad=False)

True

最后，再次为了演示，我们将使用隐式层的这个新变体来训练我们的 MNIST 网络。正如所希望的那样，该方法确实比牛顿方法的先前实现要快得多，而且要稳定得多。虽然我们再次强调指出，对于这样的设置，使用牛顿法通常不是一种合理的方法，但是当我们在后面的章节中讨论差异化优化时，非常相似的事实实际上将非常有用。

In [13]:
torch.manual_seed(0)
model = nn.Sequential(nn.Flatten(),
                      nn.Linear(784, 100),
                      TanhNewtonImplicitLayer(100, max_iter=40),
                      nn.Linear(100, 10)
                      ).to(device)
opt = optim.SGD(model.parameters(), lr=1e-1)

for i in range(10):
    if i == 5:
        opt.param_groups[0]["lr"] = 1e-2

    train_err, train_loss, train_fpiter = epoch(train_loader, model, opt, lambda x : x[2].iterations)
    test_err, test_loss, test_fpiter = epoch(test_loader, model, monitor = lambda x : x[2].iterations)
    print(f"Train Error: {train_err:.4f}, Loss: {train_loss:.4f}, Newton Iters: {train_fpiter:.2f} | " +
          f"Test Error: {test_err:.4f}, Loss: {test_loss:.4f}, Newton Iters: {test_fpiter:.2f}")

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Train Error: 0.1113, Loss: 0.4020, Newton Iters: 6.66 | Test Error: 0.0661, Loss: 0.2231, Newton Iters: 6.73


  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

Train Error: 0.0179, Loss: 0.0634, Newton Iters: 9.32 | Test Error: 0.0310, Loss: 0.1041, Newton Iters: 9.11


  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Train Error: 0.0172, Loss: 0.0612, Newton Iters: 9.80 | Test Error: 0.0301, Loss: 0.1052, Newton Iters: 9.70


## Final chapter remarks

在深入研究实际隐式模型的更加现实和多样化的世界之前，我们想突出迄今为止的成就。使用来自“传统”深度模型的很少的附加代码（并且绝对不会比传统的递归模型多得多），我们能够对一层进行编码，该层1）通过牛顿方法解决了非线性寻根问题，等同于找到无限深度网络的定点，并且2）轻松集成到自动微分工具中。这些方法的相对简单性，一旦你超越了隐式微分的一些数学符号，确实是在整个深度学习中使用隐式层的更引人注目的因素之一。

在本教程的其余部分，我们将为您提供将隐式图层应用于各种问题和设置所需的工具和背景，并提供贯穿始终的代码示例。我们希望这将使读者能够快速整合并在这个新方向上取得进展。